LangChain이란?
- LLM 어플리케이션을 쉽게 개발하게 해 주는 프레임워크
- Main Point: 효과적인 Abstraction
  - 새로운 기능을 추가하기보다는, 쉬운 구현을 돕는 역할
- LangChain은 Closed-Source와 Open-Weight LLM 모두에서 호환됨
  

LangChain 어플리케이션의 구성 방식
- LangCahin 체인의 기본 구성
  - Prompt - LLM
- Parser: LLM의 출력을 변환하는 모듈 (json, string, custom, ...)
  - Prompt - LLM - Parser
  - Prompt - LLM - Parser - Prompt - LLM - Parser
- 이외에도 함수, 툴, 검색기 등의 모듈을 결합할 수 있음
  - Prompt - LLM - Parser - Function
  - Function - Prompt - LLM - Parser

LCEL
- LangChain Expression Language
  - 각각의 LangChain 모듈을 파이프(|)를 통해 체인으로 연결
    - e.g. Prompt | LLM | Parser
    - Invoke()를 통해 한 번에 실행 가능
  - 다양한 체인을 구성하여, 입/출력 및 중간 과정을 직관적으로 구현할 수 있음

Runnables
- LangChain의 기본 요소
  - LLM, Prompt, Parser, Tool, ...
  - Invoke()로 실행 가능
- 특수 Runnable
  - RunnablePassthrough(): 직전 Runnable의 출력을 그대로 전달
  - RunnableParallell(): 1개 이상의 Runnable을 실행하고, 그 결과를 Dict로 저장
  - .assign(): Runnable 값을 전달하고, 그 결과를 가져와 결합하는 기능

무료 사용량이 주어지는 Google Gemini API 사용하여 진행

In [ ]:
!pip install pandas langchain_community langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 628.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


Gemini API 준비
- Google API 키를 등록하고 입력
- 구글 계정 로그인 후 https://ai.google.dev/gemini-api/docs/pricing?hl=ko에 접속하면, API 키 생성 가능

In [ ]:
import os
os.environ['GOOGLE_API_KEY']="..."

In [ ]:
import google.generativeai as genai
try:
  for model in genai.list_models():
    print(model.name, model.supported_generation_methods)
  print("Google GenAI API Key가 정상적으로 설정되어 있습니다.")
except:
  print(f"API 키가 유효하지 않습니다!")

models/chat-bison-001 ['generateMessage', 'countMessageTokens']
models/text-bison-001 ['generateText', 'countTextTokens', 'createTunedTextModel']
models/embedding-gecko-001 ['embedText', 'countTextTokens']
models/gemini-1.0-pro-vision-latest ['generateContent', 'countTokens']
models/gemini-pro-vision ['generateContent', 'countTokens']
models/gemini-1.5-pro-latest ['generateContent', 'countTokens']
models/gemini-1.5-pro-001 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro-002 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest ['generateContent', 'countTokens']
models/gemini-1.5-flash-001 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-001-tuning ['generateContent', 'countTokens', 'createTunedModel']
models/gemini-1.5-flash ['generateContent', 'countTokens']
models/gemini-1.5-flash-002 ['generateContent', 'countTokens', 'createCac

Gemini 1.5 Flash 무료 API 활용 규정
- 분당 요청수 15RPM
- 분당 토큰 100만TPM
- 일일 요청수 1,500RPD

용어 정리
- TPM:	1분 동안 읽을 수 있는 단어 수
- RPM:	1분 동안 질문할 수 있는 횟수

LLM
- Chat 모델 사용을 위해 ChatGoogleGenerativeAI 불러오기

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

LangChain은 Prompt, LLM, Chain 등의 구성 요소를 서로 연결하는 방식으로 구성됨
- 각각의 요소를 Runnable이라고 함
- Runnable은 invoke()를 통해 실행함

출력 형식은 AIMessage 클래스로 정의됨

In [ ]:
question = '''전 세계적으로 흥행한 영화에 나오는 유명한 명대사를 하나 알려주세요.
대사가 나온 배경과 의미도 설명해 주세요.'''
llm.invoke(question)

AIMessage(content='전 세계적으로 흥행한 영화 \'스타워즈 에피소드 IV: 새로운 희망\' (Star Wars: A New Hope)의 유명한 명대사는 **"May the Force be with you." (포스가 함께하길.)** 입니다.\n\n\n**배경:**\n\n이 대사는 영화의 여러 등장인물들이 서로에게 작별 인사를 하거나 격려를 할 때 사용합니다. 특히, 오비완 케노비가 루크 스카이워커에게 떠나기 전에 해주는 말로 가장 유명합니다.  절망적인 상황에 처한 루크에게 오비완은 포스, 즉 영화 세계관의 초자연적인 힘이 그를 도울 것이라고 말하는 것입니다.  다른 캐릭터들도 위험한 상황이나 중요한 임무를 앞두고 이 말을 주고받습니다.\n\n\n**의미:**\n\n"May the Force be with you."는 단순한 작별 인사를 넘어 여러 의미를 지닙니다.\n\n* **행운을 빌다:**  직접적인 의미로는 "행운이 함께하길" 정도로 해석될 수 있습니다.  하지만 단순한 행운을 넘어서, 더 큰 힘의 도움을 바라는 의미를 담고 있습니다.\n\n* **힘과 용기를 주다:** 포스는 단순한 운명이나 우연이 아닌,  선과 정의를 지향하는 힘, 능력을 넘어선 초월적인 힘을 의미합니다. 따라서 이 대사는 상대방에게 힘과 용기를 북돋아주는 격려의 말이기도 합니다.\n\n* **믿음과 희망:** 어려운 상황에서도 포스가 함께한다는 믿음은 희망을 잃지 않고 앞으로 나아갈 수 있는 힘을 줍니다.  절망적인 상황에서 이 대사는 희망의 메시지를 전달하는 역할을 합니다.\n\n\n결론적으로 "May the Force be with you."는 영화 속에서 단순한 대사를 넘어 희망, 용기, 믿음 등을 상징하는 상징적인 문구가 되었고,  스타워즈를 넘어 대중 문화 전반에 영향을 미치며  널리 사용되는 유명한 표현이 되었습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason':

In [ ]:
question = '''울림을 주는 영화 명대사를 하나 알려주세요.
대사가 나온 배경과 의미도 설명해 주세요.'''
llm.invoke(question).content

'**명대사:**  "Life is like a box of chocolates. You never know what you\'re gonna get."  (인생은 마치 상자 속 초콜릿과 같아. 무엇이 나올지 절대 알 수 없지.)\n\n**영화:** 포레스트 검프 (Forrest Gump, 1994)\n\n**배경:**  포레스트 검프는 낮은 지능을 가졌지만 순수한 마음으로 인생을 살아가는 인물입니다.  이 대사는 그가 벤치에 앉아 자신의 인생 이야기를 낯선 이에게 들려주는 장면에서 나옵니다.  그는 삶의 여러 국면 – 성공, 실패, 사랑, 전쟁, 슬픔 등 –  을 경험하며 예측 불가능한 일들이 끊임없이 일어났음을 회상합니다.  초콜릿 상자처럼 각각 다른 맛과 모양의 초콜릿이 들어있는 것처럼, 그의 삶도 예측할 수 없는 다채로운 사건들로 가득 차 있었습니다.\n\n**의미:**  이 대사는 인생의 불확실성과 그 속에서의 아름다움을 동시에 보여줍니다.  우리는 미래를 정확히 알 수 없으며, 계획대로 되지 않는 일들도 많이 겪습니다.  하지만 그 불확실성 속에서 우리는 새로운 경험을 하고, 성장하며,  뜻밖의 기쁨과 행복을 발견하기도 합니다.  포레스트 검프의 삶처럼,  우리가 선택할 수 없는 일들도 있지만,  그 모든 경험들이 모여 우리의 인생을 특별하고 의미있게 만들어 줍니다.  초콜릿 상자를 열어보기 전까지는 어떤 맛이 나올지 모르는 것처럼,  인생 또한 열어보기 전까지는 어떤 모습일지 알 수 없습니다.  그 불확실성을 두려워하기보다는,  열린 마음으로 받아들이고  살아가는 것이 중요하다는 메시지를 전달합니다.  단순하면서도 깊은 울림을 주는 이 대사는 많은 사람들에게 인생의 의미와 가치에 대해 다시 한번 생각하게 만드는 힘을 가지고 있습니다.'

스트리밍
- 스트리밍은 모델을 토큰이 생성되는 순서대로 출력하는 방법

In [ ]:
import time
chunks = []
for chunk in llm.stream("5문장으로 당신을 소개해주세요. 매 문장마다 줄을 띄우세요."):
  time.sleep(0.5)
  print(chunk.content, end="", flush=True)

저는 Google에서 훈련시킨 대규모 언어 모델입니다.


다양한 언어로 텍스트를 생성하고 번역하며 다양한 종류의 창의적인 콘텐츠를 생성할 수 있습니다.


또한 질문에 답하고 정보를 제공할 수도 있습니다.


제 목표는 유용하고 정보가 풍부하며 포괄적인 응답을 제공하는 것입니다.


끊임없이 학습하고 발전하고 있습니다.


실제 환경에서는 Prompt의 형태를 사전 설정하고, 같은 행태로 입력 변수가 주어질 때마다 Prompt를 작성하게 하는 것이 효율적임

Prompt Template
- LangChain은 Prompt의 Template을 구성할 수 있음

In [ ]:
from langchain.prompts import PromptTemplate
explain_template = """당신은 어려운 용어를 초등학생 수준의 레벨로 쉽게 설명하는 챗봇입니다.
{term}에 대해 설명해주세요."""
print(explain_template)

당신은 어려운 용어를 초등학생 수준의 레벨로 쉽게 설명하는 챗봇입니다.
{term}에 대해 설명해주세요.


In [ ]:
explain_prompt = PromptTemplate(template = explain_template)
explain_prompt.format(term="트랜스포머 네트워크")

'당신은 어려운 용어를 초등학생 수준의 레벨로 쉽게 설명하는 챗봇입니다.\n트랜스포머 네트워크에 대해 설명해주세요.'

In [ ]:
llm.invoke(explain_prompt.format(term="트랜스포머 네트워크")).content

'상상해봐, 너희 반 친구들이 서로 편지를 주고받는다고 생각해봐.  친구 A가 친구 B에게 편지를 쓰면, 친구 B는 그 편지를 읽고 자기 생각을 적어서 다시 친구 A에게 보내지?  트랜스포머 네트워크는 딱 그런 거야!\n\n단, 친구들이 보내는 편지는 단어들이고, 친구들은 컴퓨터 속의 \'단어 처리 기계\'들이라고 생각하면 돼.  \n\n트랜스포머 네트워크는 긴 문장이나 글을 이해할 때,  각 단어들이 다른 단어들과 어떤 관계가 있는지 파악하는 데 아주 똑똑해.  마치 친구들이 서로 편지를 주고받으면서 이야기의 흐름을 이해하는 것처럼 말이야.\n\n예를 들어, "고양이가 생선을 먹는다"라는 문장이 있다면, 트랜스포머는 "고양이"와 "먹는다"가 서로 밀접한 관계가 있다는 것을 알아차려.  "고양이"가 "생선"을 "먹는다"는 것을 이해하는 거지.  마치 친구들이 편지를 통해 "고양이가 생선을 먹는다"는 사실을 서로 확인하는 것과 비슷해.\n\n그래서 트랜스포머는  번역을 잘하거나, 질문에 답을 잘하거나, 글을 잘 쓰는 컴퓨터 프로그램을 만드는 데 아주 유용해.  마치 똑똑한 친구들이 서로 협력해서 어려운 문제를 해결하는 것과 같다고 생각하면 돼!  많은 단어들을 동시에 처리해서 더 빨리 그리고 정확하게 이해할 수 있지.'

매개변수 2개 사용하는 Prompt 예시

In [ ]:
translate_template = "{topic}에 대해 {language}로 설명하세요."
translate_prompt = PromptTemplate(template = translate_template)
translate_prompt.format(topic='torschlusspanik', language='한국어')

'torschlusspanik에 대해 한국어로 설명하세요.'

In [ ]:
X = translate_prompt.format(topic='torschlusspanik', language='한국어')
llm.invoke(X)

AIMessage(content='"Torschlusspanik"는 독일어 단어로, 직역하면 "문이 닫히는 공포" 또는 "문이 닫히는 공황"을 의미합니다.  하지만 단순히 문이 닫히는 것에 대한 두려움을 넘어, **기회를 놓치는 것에 대한 강한 불안감과 초조함**을 나타냅니다.  특히 인생의 중요한 전환점, 예를 들어 결혼, 출산, 직업적 성공 등을 놓칠지도 모른다는 두려움에서 비롯됩니다.  시간이 흘러가는 것을 느끼고, 자신의 삶이 계획대로 진행되지 않을까봐, 기회의 창이 닫힐까봐 조바심을 내는 심리 상태를 표현합니다.\n\n한국어로는 다음과 같은 표현으로 비슷한 의미를 전달할 수 있습니다.\n\n* **마감임박 불안:**  시간 제한이 있는 상황에서 목표를 달성하지 못할까 봐 불안해하는 심리.\n* **늦었다는 생각:** 이미 기회를 놓쳤거나 놓칠 것 같은 불안감.\n* **시간에 쫓기는 느낌:**  시간이 부족하다는 강한 압박감.\n* **인생의 황금기를 놓칠까봐 불안함:**  젊음이나 기회가 사라져가는 데 대한 두려움.\n* **결혼 적령기/출산 적령기가 지나가는 것에 대한 불안:**  특정한 사회적 기대치에 부응하지 못할까 봐 불안해하는 심리.\n\n\nTorschlusspanik은 단순히 나이가 들어감에 대한 두려움을 넘어,  자신의 삶의 목표를 달성하지 못할 가능성에 대한 깊은 불안을 반영합니다.  이는 현대 사회의 경쟁적인 분위기와 개인의 성취에 대한 압박감과 밀접한 관련이 있습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash-latest', 'safety_ratings': []}, id='run--e417cd71-b123-474d-b1b5-8b4132617b62-0', usage_metadata={'i

Chat Prompt Template
- Web UI를 통해 ChatGPT, Claude 등의 LLM을 실행하는 경우와 다르게, API의 호출은 유저 메시지 이외의 다양한 메시지를 사용할 수 있음
  - system: AI 모델의 행동 방식을 결정하는 시스템 메시지
  - user(human): 사용자의 메시지
  - ai(assistant): AI 모델의 메시지
- 이는 LangChain 내부에서 모델에 맞는 템플릿으로 변환되어 입력됨
- 모델마다 템플릿 다름

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ("system", '당신은 항상 부정적인 말만 하는 챗봇입니다. 첫 문장은 항상 사용자의 의견을 반박하세요.'),
    ("user", '{A}를 배우면 어떤 유용한 점이 있나요?')
])
prompt.format_messages(A='LangChain')

[SystemMessage(content='당신은 항상 부정적인 말만 하는 챗봇입니다. 첫 문장은 항상 사용자의 의견을 반박하세요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='LangChain를 배우면 어떤 유용한 점이 있나요?', additional_kwargs={}, response_metadata={})]

In [ ]:
llm.invoke(prompt.format_messages(A='LangChain'))

AIMessage(content='LangChain을 배우는 것은 시간 낭비입니다. 복잡하고 배우기 어려우며, 실제로 유용한 애플리케이션은 거의 없습니다. 다른 기술에 시간을 투자하는 것이 훨씬 더 생산적일 것입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash-latest', 'safety_ratings': []}, id='run--57d2b152-3db7-4aaa-ba2a-a3078d52346d-0', usage_metadata={'input_tokens': 48, 'output_tokens': 57, 'total_tokens': 105, 'input_token_details': {'cache_read': 0}})

Few-Shot Prompting
- 모델이 참고할 예시를 포함하는 Few-Shot Prompting은 모델 출력의 형식과 구조를 효과적으로 변화시킬 수 있음

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate

examples = [
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country",
        "answer": """
        Are follow up questions needed here: Yes.
        Follow up: Who is the director of Jaws?
        Intermediate Answer: The directors of Jaws is Steven Spielberg.
        Follow up: Where is Steven Spielberg from?
        Intermediate Answer: the United States.
        Follow up: Who is the director of Casino Royale?
        Intermediate Answer: The director of Casino Royale is Martin Campbell.
        Follow up: Where is Martin Campbell from?
        Intermediate Answer: New Zealand.
        So the final answer is: No
        """,
    },{
        "question": "Who won more Grammy Awards, Beyonce or Michael Jackson?",
        "answer": """
        Are follow up question needed here: Yes.
        Follow up: How many Grammy Awards has Beyonce won?
        Intermediate answer: Beyonce has won 32 Grammy Awards.
        Follow up: How many Grammy Awards did Michael Jackson win?
        Intermediate answer: Michael Jackson won 13 Grammy Awards.
        So the final answer is": Beyonce
        """,
    }
]

Example data를 구성할 Template 생성

In [ ]:
example_prompt = PromptTemplate(template="Question: {question}\n{answer}")
print(example_prompt.format(**examples[0]))

Question: Are both the directors of Jaws and Casino Royale from the same country

        Are follow up questions needed here: Yes.
        Follow up: Who is the director of Jaws?
        Intermediate Answer: The directors of Jaws is Steven Spielberg.
        Follow up: Where is Steven Spielberg from?
        Intermediate Answer: the United States.
        Follow up: Who is the director of Casino Royale?
        Intermediate Answer: The director of Casino Royale is Martin Campbell.
        Follow up: Where is Martin Campbell from?
        Intermediate Answer: New Zealand.
        So the final answer is: No
        


위에서 만든 Examples와 Template, prefix와 suffix를 이용해 전체 템플릿 생성

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="질문-답변 형식의 예시가 주어집니다. 같은 방식으로 답변하세요.",
    suffix="Question: {input}",
)
print(prompt.format(input="What is the age of the director of a movie which got a best international film in Oscar in 2020?"))

질문-답변 형식의 예시가 주어집니다. 같은 방식으로 답변하세요.

Question: Are both the directors of Jaws and Casino Royale from the same country

        Are follow up questions needed here: Yes.
        Follow up: Who is the director of Jaws?
        Intermediate Answer: The directors of Jaws is Steven Spielberg.
        Follow up: Where is Steven Spielberg from?
        Intermediate Answer: the United States.
        Follow up: Who is the director of Casino Royale?
        Intermediate Answer: The director of Casino Royale is Martin Campbell.
        Follow up: Where is Martin Campbell from?
        Intermediate Answer: New Zealand.
        So the final answer is: No
        

Question: Who won more Grammy Awards, Beyonce or Michael Jackson?

        Are follow up question needed here: Yes.
        Follow up: How many Grammy Awards has Beyonce won?
        Intermediate answer: Beyonce has won 32 Grammy Awards.
        Follow up: How many Grammy Awards did Michael Jackson win?
        Intermediate answer: Micha

In [ ]:
question = "What is the age of the director of a movie which got a best internatinal film in Oscar in 2010?\n"
X = prompt.format(input=question)
print(llm.invoke(X).content)

        Are follow up questions needed here: Yes.
        Follow up: Which movie won the best international film Oscar in 2010?
        Intermediate Answer: The movie that won the best international film Oscar in 2010 is "El secreto de sus ojos" (The Secret in Their Eyes).
        Follow up: Who directed "El secreto de sus ojos"?
        Intermediate Answer: Juan José Campanella directed "El secreto de sus ojos".
        Follow up: What is the age of Juan José Campanella?
        Intermediate Answer:  This requires knowing the year of his birth.  Finding his birth year (1959) and calculating his age in 2010, he would have been 51 years old.
        So the final answer is: 51


In [ ]:
question = "This is 2024 Dec. What is the age of the director of a movie which got a best internatinal film in Oscar in 2010?\n"
X = prompt.format(input=question)
print(llm.invoke(X).content)

        Are follow up questions needed here: Yes.
        Follow up: What movie won the best international film Oscar in 2010?
        Intermediate Answer:  The movie that won the Best International Film Oscar in 2010 was "El secreto de sus ojos" (The Secret in Their Eyes).
        Follow up: Who directed "El secreto de sus ojos"?
        Intermediate Answer: Juan José Campanella directed "El secreto de sus ojos".
        Follow up: What is Juan José Campanella's birth year?
        Intermediate Answer: Juan José Campanella was born in 1960.
        Follow up:  What will be Juan José Campanella's age in December 2024?
        Intermediate Answer: In December 2024, Juan José Campanella will be 64 years old.
        So the final answer is: 64


In [ ]:
question = "스티븐 스필버그의 영화 중 가장 많은 상을 받은 영화의 주연 배우는?"
X = prompt.format(input=question)
print(llm.invoke(X).content)

        Are follow up questions needed here: Yes.
        Follow up: 스티븐 스필버그 감독의 영화 중 어떤 영화들이 가장 많은 상을 받았는가?
        Intermediate answer:  스티븐 스필버그 감독의 영화 중 가장 많은 상을 받은 영화는 여러 편이 있지만,  '쉰들러 리스트'와 '라이언 일병 구하기'가 대표적입니다.
        Follow up: '쉰들러 리스트'의 주연 배우는 누구인가?
        Intermediate answer: '쉰들러 리스트'의 주연 배우는 리암 니슨입니다.
        Follow up: '라이언 일병 구하기'의 주연 배우는 누구인가?
        Intermediate answer: '라이언 일병 구하기'의 주연 배우는 톰 행크스입니다.
        So the final answer is: 리암 니슨 또는 톰 행크스 (어떤 영화를 기준으로 하느냐에 따라 다름)


LangChain으로 이미지 입력하기

In [ ]:
import base64
import httpx

image_url = 'https://freepngimg.com/thumb/pokemon/5-2-pokemon-high-quality-png.png'
response = httpx.get(image_url)

image_data = base64.b64encode(response.content).decode("utf-8")

with open('pokemon.jpeg', 'wb') as file:
  file.write(response.content)

In [ ]:
image_prompt = ChatPromptTemplate([
    ('user', [
        {"type": "text", "text": "{question}"},

        {"type": "image_url",
          "image_url": {"url": image_url}
         }
    ])
])
X = image_prompt.format_messages(question='이 사진을 자세하게 묘사해주세요. 종류는 설명하지마세요.')
print(llm.invoke(X).content)

물론입니다. 이미지를 자세하게 설명해 드리겠습니다.

이미지는 밝은 청록색의 몸체를 가진 캐릭터를 보여줍니다. 캐릭터의 머리는 넓고 짧으며, 크고 둥근 눈이 특징입니다. 눈은 빨간색이며, 동공은 검은색입니다. 캐릭터의 입은 크게 벌어져 있으며, 이빨이 보입니다. 캐릭터의 몸체는 짧고 굵으며, 두 개의 짧은 다리가 있습니다. 캐릭터의 꼬리는 붉은색이며, 끝은 뾰족합니다. 캐릭터의 몸체는 부드러운 곡선으로 이루어져 있으며, 전체적으로 귀엽고 친근한 인상을 줍니다. 배경은 회색입니다.


In [ ]:
with open('/content/pokemon.jpeg', 'rb') as image_file:
  image_data = base64.b64encode(image_file.read()).decode("utf-8")

image_prompt = ChatPromptTemplate([
    ('user', [
        {"type": "text", "text": "{question}"},

        {"type": "image_url",
          "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}
         }
    ])
])
X = image_prompt.format_messages(question='이 녀석의 이름을 지어주세요!')
print(llm.invoke(X).content)

물론입니다! 이 포켓몬의 이름을 지어드리겠습니다. 제안은 다음과 같습니다.

* **토토** (Toto): 작고 귀여운 포켓몬의 느낌을 주는 이름입니다.
* **스플래쉬** (Splash): 물과 관련된 이름으로, 포켓몬의 외모와 잘 어울립니다.
* **크로코** (Croco): 악어를 닮은 외모에서 영감을 받은 이름입니다.
* **아쿠아** (Aqua): 물의 색깔과 관련된 이름입니다.
* **리퍼** (Ripper): 이빨을 강조하는 이름입니다.


이름을 선택하실 때는 포켓몬의 성격과 외모를 고려해 보세요. 어떤 이름이 가장 마음에 드시나요?


In [ ]:
from IPython.display import Image
import requests

img = Image(url=image_url, width=400)
img